## **ECG Diagnosis Code**

This code is based on the code developed here: https://doi.org/10.1038/s41467-020-15432-4

**Define Libraries**

In [1]:
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report
import numpy as np
import h5py
import math
import pandas as pd
from tensorflow.keras.utils import Sequence
import numpy as np
import os

In [2]:
cwd = os.getcwd()

**Load the data**

In [3]:
def ProcessLabels(data_info: pd.DataFrame, train_ids: np.ndarray, abnormalities: list):
    good_rows = []
    bad_rows = []
    labels = np.array([])
    for ii, id in enumerate(train_ids):
        if id in data_info['exam_id'].to_numpy():
            good_rows.append(ii)
            data_row = data_info.iloc[np.where(data_info['exam_id'].to_numpy() == id)[0][0]]
            labels = np.append(labels, data_row[abnormalities].values.tolist())
        else:
            bad_rows.append(ii)

    labels = np.reshape(labels, (len(good_rows), len(abnormalities)))
    
    return labels.astype('int64'), good_rows

#Order is based on test set csv
abnormalities = ['1dAVb','RBBB', 'LBBB', 'SB',  'AF', 'ST']

#Load in training data
path_to_hdf5 = cwd + '\\data\\train\\exams_part17.hdf5'
dataset_name = 'tracings'
path_to_csv = cwd + '\\data\\train\\exams.csv'

#Process the exams data
f = h5py.File(path_to_hdf5, "r")
tracings_train = f[dataset_name][()]
train_ids = f['exam_id'][()]
f.close()
labels_train, good_rows = ProcessLabels(pd.read_csv(path_to_csv), train_ids, abnormalities)
tracings_train = tracings_train[good_rows]

#Load in test data
path_to_hdf5 = cwd + '\\data\\test\\ecg_tracings.hdf5'
dataset_name = 'tracings'
path_to_csv = cwd + '\\data\\test\\gold_standard.csv'

labels_test = pd.read_csv(path_to_csv).values
f = h5py.File(path_to_hdf5, "r")
tracings_test = f[dataset_name][()]
f.close()

## Current Model

**Define the NN model**

In [4]:
class ResidualUnit(object):
    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1]
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2]
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]


def get_model(n_classes, last_layer='sigmoid'):
    kernel_size = 16
    kernel_initializer = 'he_normal'
    signal = Input(shape=(4096, 12), dtype=np.float32, name='signal')
    x = signal
    x = Conv1D(64, kernel_size, padding='same', use_bias=False,
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x, y = ResidualUnit(1024, 128, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, x])
    x, y = ResidualUnit(256, 196, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, y = ResidualUnit(64, 256, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, _ = ResidualUnit(16, 320, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x = Flatten()(x)
    diagn = Dense(n_classes, activation=last_layer, kernel_initializer=kernel_initializer)(x)
    model = Model(signal, diagn)
    return model


**Load Parameters**

Loading the parameters for the model that were found in the paper. We will call this our first model

They trained 10 NN with different initializations. The choose the model to use based on the median micro average persion (mAP = 0.951). They had to choose the one right above the median since 10 is even so they can't take the median execution

In [5]:
path_to_model = cwd + '\\model\\model.hdf5'

loss = 'binary_crossentropy'
lr = 0.001
batch_size = 64
opt = Adam(lr)

model_1 = load_model(path_to_model, compile=False)
model_1.compile(loss=loss, optimizer=opt)

**Data Fromatting**

Here is the class for transforming the data into the proper format

In [6]:
class ECGSequence(Sequence):
    @classmethod
    def get_seq(cls, tracings: np.ndarray, labels: np.ndarray=None, batch_size=8):
        train_seq = cls(tracings, labels, batch_size)
        return train_seq

    def __init__(self, tracings:np.ndarray, labels:np.ndarray=None, batch_size:int=8):
        if labels is None:
            self.y = None
        else:
            self.y = labels
        # Get tracings
        self.x = tracings
        self.batch_size = batch_size
        self.sample_count = tracings.shape[0]

    @property
    def n_classes(self):
        return self.y.shape[1]

    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = start + self.batch_size
        if self.y is None:
            return np.array(self.x[start:end, :, :])
        else:
            return np.array(self.x[start:end, :, :]), np.array(self.y[start:end])

    def __len__(self):
        return math.ceil(self.sample_count / self.batch_size)


**Training Function**

We will also train the model with the data accessible for better comparison with the simplified model. We will call this the second model

For sake of computational resources and time, the second model was only trained once instead of trained 10 times and then taking the model based on the median mAP

In [7]:
class MyCNN:
    def __init__(self, loss, opt, verbose):
        # Optimization settings
        self.callbacks = [ReduceLROnPlateau(monitor='val_loss',
                            factor=0.1,
                            patience=7,
                            min_lr=lr / 100),
                            EarlyStopping(monitor='val_loss', 
                            patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                            min_delta=0.00001)]

        self.loss = loss
        self.optimizer = opt
        self.verbose = verbose
        
        # Save the BEST and LAST model
        '''callbacks += [ModelCheckpoint('./backup_model_last.hdf5'),
        ModelCheckpoint('./backup_model_best.hdf5', save_best_only=True)]'''

    def train(self, train_seq, val_seq):
        self.model = get_model(train_seq.n_classes)
        self.model.compile(loss=self.loss, optimizer=self.optimizer)
        # Train neural network
        self.model.fit(train_seq,
            epochs=70,
            initial_epoch=0,  # If you are continuing a interrupted section change here
            callbacks=self.callbacks,
            validation_data=val_seq,
            verbose=self.verbose)

    def predict(self, test_seq):
        return self.model.predict(test_seq,  verbose=self.verbose)




## Simplified Models

Need to choose what model I want

Going to have to use something like random forest because I need a multi-label classifier, or I can use sklearn.multioutput.MultiOutputClassifier and use any classifier

I think all of the data for all 12 leads is the set of features for each sample

**Define Models**

In [8]:
#TODO: Tune the hyperparameters
class RF_Model:
    def __init__(self, verbose = 1):
        self.model = RandomForestClassifier(verbose=verbose)

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X,y)

    def predict(self, X):
        return self.model.predict(X)

class LR_model:
    def __init__(self, verbose = 1):
        self.model = MultiOutputClassifier(LogisticRegression(verbose=verbose))

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X, y)

    def predict(self, X: np.ndarray):
        return self.model.predict(X)

**Data Fromatting**

PCA for the simplified models

In [9]:
class PCA_Transform:
    def __init__(self, r:int):
        self.PCA_instance = PCA(n_components=r)
    
    def _flattenData(self, X:np.ndarray):
        return np.hstack(X)

    def _processData(self, X: np.ndarray):
        X_flat = self._flattenData(X)
        self.preprocess = StandardScaler()
        self.preprocess.fit(X_flat)

    def FitData(self, X: np.ndarray):
        self._processData(X)
        self.PCA_instance.fit(self.preprocess.transform(self._flattenData(X)))

    def TransformData(self, X_train: np.ndarray, X_test: np.ndarray):
        X_train = self.preprocess.transform(self._flattenData(X_train))
        X_test = self.preprocess.transform(self._flattenData(X_test))
        return self.PCA_instance.transform(X_train), self.PCA_instance.transform(X_test)

## K-Fold

K-fold procedure for validation of the models

They use a validation set of 2% so something to think about

They didn't round for the outputs, seems to be a threshold in which they consider it to occur

They used precision-recall curves for things, but in total found precision, recall, specificity and F1 score

**Process Outputs**

Function for processing the output probabilities from the CNNs

In [14]:
#TODO: Need to optimize these

#Thinking I could optimize them with the model 1 for each test set
# could do this with minimizing the sum of the absolute difference between the pred and true

def FindThresholds(y_pred: np.ndarray, y_true: np.ndarray):
    '''Optimize the thresholds based on the F1-score for the model 1 in the test set'''

    start = 0.05
    end = 0.8
    step = 0.001
    threshold_options = np.linspace(start,end,int((end-start)/step))
    optimal_thresholds = np.array([])

    for ii in range(y_pred.shape[1]):
        #use the np greater for a list of thresholds and then find max F1-score and index it back to the thresholds
        F1_scores = [f1_score(y_true=y_true[:,ii], y_pred=ProcessOutputs(y_pred[:,ii], threshold)) for threshold in threshold_options]
        optimal_thresholds = np.append(optimal_thresholds, threshold_options[np.argmax(F1_scores)])

    return optimal_thresholds


def ProcessOutputs(outputs: np.ndarray, thresholds: np.ndarray):

    threshold_check = np.array([np.greater_equal(sample, thresholds) for sample in outputs])
    threshold_outputs = threshold_check.astype(int)
    
    return threshold_outputs

**Metrics Function**

Making a function to be able to call all of the metrics each fold

In [11]:
'''scores = {'Precision': {'Model_1': np.array([]), 'Model_2': np.array([]), 'Model_3': np.array([]), 'Model_4': np.array([])},\
        'Recall': {'Model_1': np.array([]), 'Model_2': np.array([]), 'Model_3': np.array([]), 'Model_4': np.array([])},\
        'F1': {'Model_1': np.array([]), 'Model_2': np.array([]), 'Model_3': np.array([]), 'Model_4': np.array([])}}'''

def Score_initator():

        scores = dict()
        metrics = ['Precision', 'Recall', 'F1']
        models = ['m1', 'm2', 'm3', 'm4']
        for ii, metric in enumerate(metrics):
                scores[metric] = dict()
                for jj, model in enumerate(models):
                        scores[metric][model] = dict()
                        for zz, abnomality in enumerate(abnormalities):
                                scores[metric][model][abnomality] = np.array([])
        
        return scores


#TODO: Add specificity and add the ability to pull these for each class, maybe can just make an index and label encoder for the metrics
#with each numpy array being 2d
def Find_metrics(scores: dict, model_name: str, y_true: np.ndarray, y_pred: np.ndarray):
        
        def _findProperScore(metric, y_true, y_pred):
                if metric == 'Precision':
                        return precision_score(y_true, y_pred)  
                elif metric == 'Recall':
                        return recall_score(y_true, y_pred)
                else:
                        return f1_score(y_true, y_pred)

        for jj, metric in enumerate(scores.keys()):
                for ii, cardio_class in enumerate(scores[metric][model_name].keys()):

                        scores[metric][model_name][cardio_class] = np.append(scores[metric][model_name][cardio_class], \
                                _findProperScore(metric, y_true[:,ii], y_pred[:,ii]))

        return scores

In [13]:
kf = KFold(n_splits=3, shuffle=True)

scores = Score_initator()

#Initilaize the models that need to be trained
model_verbose = 1
model_2 = MyCNN(loss, opt, verbose = model_verbose)
'''model_3 = RF_Model(verbose = model_verbose)
model_4 = LR_model(verbose = model_verbose)'''

#PCA initlization
'''PCA_transformer = PCA_Transform(r = 60)'''

#TODO: Actually make a validation set for the early stopping
#Also there is no regularization in the layers so maybe can add that
for train_index, test_index in kf.split(X = tracings_train[:,1,1], y = labels_train):

        X_train, X_test = tracings_train[train_index,:,:], tracings_train[test_index,:,:]
        y_train, y_test = labels_train[train_index], labels_train[test_index]

        #Put data in sequence for models 1 and 2 (CNN)
        train_seq = ECGSequence.get_seq(X_train, y_train, batch_size=64)

        test_seq = ECGSequence.get_seq(X_test, y_test, batch_size=64)

        #Transform data with PCA for models 3 and 4
        #TODO: Fix to do for each channel seperatley
        '''PCA_transformer.FitData(X_train)
        PCA_X_train, PCA_X_test = PCA_transformer.TransformData(X_train, X_test)'''

        #Train models
        #Here the validation seq is just the test seq since we are using a k-fold analysis
        print('\n-------------------Training model 2----------------------')
        #model_2.train(train_seq, val_seq=test_seq)
        '''print('\n-------------------Training model 3----------------------')
        model_3.train(X = PCA_X_train, y = y_train)
        print('\n-------------------Training model 4----------------------')
        model_4.train(X = PCA_X_train, y = y_train)'''

        #Test models - have to round the outputs of the script (TODO check what they do)
        print('\n-------------------Testing model 1----------------------')
        model_1_predict = np.round(model_1.predict(test_seq, verbose=model_verbose)).astype(int)
        print('\n-------------------Testing model 2----------------------')
        #model_2_predict = np.rint(model_2.predict(test_seq), dtype=int)
        '''print('\n-------------------Testing model 3----------------------')
        model_3_predict = model_3.predict(PCA_X_train)
        print('\n-------------------Testing model 4----------------------')
        model_4_predict = model_4.predict(PCA_X_train)'''

        

        #Find metrics
        #Call the function each time and then can average for each class after

        scores = Find_metrics(scores, 'm1', y_test, model_1_predict)
        #scores = Find_metrics(scores, 'm2', y_test, model_2_predict)
        '''scores = Find_metrics(scores, 'm3', y_test, model_3_predict)
        scores = Find_metrics(scores, 'm4', y_test, model_3_predict)'''



-------------------Training model 2----------------------

-------------------Testing model 1----------------------
31/31 [==============================] - 20s 652ms/step

-------------------Testing model 2----------------------

-------------------Training model 2----------------------

-------------------Testing model 1----------------------
31/31 [==============================] - 20s 631ms/step

-------------------Testing model 2----------------------

-------------------Training model 2----------------------

-------------------Testing model 1----------------------
31/31 [==============================] - 20s 638ms/step

-------------------Testing model 2----------------------


## Testing

**Train Models**

Train all the models that need to be trained on the entire training set

In [ ]:
train_seq = ECGSequence.get_seq(tracings_train, labels_train, batch_size=64)

        

**Test Models**

Test all the models with the test set

In [16]:
Test_scores = Score_initator()
test_seq = ECGSequence.get_seq(tracings_test, labels_test, batch_size=64)

#Predict with the models
print('\n-------------------Testing model 1----------------------')
model_1_predict = model_1.predict(test_seq, verbose=1)
optimal_thresholds = FindThresholds(model_1_predict, labels_test)
model_1_predict = ProcessOutputs(model_1_predict, optimal_thresholds)
print('\n-------------------Testing model 2----------------------')
#model_2_predict = np.rint(model_2.predict(test_seq), dtype=int)
'''print('\n-------------------Testing model 3----------------------')
model_3_predict = model_3.predict(PCA_X_train)
print('\n-------------------Testing model 4----------------------')
model_4_predict = model_4.predict(PCA_X_train)'''


Test_scores = Find_metrics(Test_scores, 'm1', labels_test, model_1_predict)
print(Test_scores)


-------------------Testing model 1----------------------
13/13 [==============================] - 8s 616ms/step

-------------------Testing model 2----------------------
{'Precision': {'m1': {'1dAVb': array([0.88888889]), 'RBBB': array([0.91891892]), 'LBBB': array([1.]), 'SB': array([0.78947368]), 'AF': array([0.84615385]), 'ST': array([0.92307692])}, 'm2': {'1dAVb': array([], dtype=float64), 'RBBB': array([], dtype=float64), 'LBBB': array([], dtype=float64), 'SB': array([], dtype=float64), 'AF': array([], dtype=float64), 'ST': array([], dtype=float64)}, 'm3': {'1dAVb': array([], dtype=float64), 'RBBB': array([], dtype=float64), 'LBBB': array([], dtype=float64), 'SB': array([], dtype=float64), 'AF': array([], dtype=float64), 'ST': array([], dtype=float64)}, 'm4': {'1dAVb': array([], dtype=float64), 'RBBB': array([], dtype=float64), 'LBBB': array([], dtype=float64), 'SB': array([], dtype=float64), 'AF': array([], dtype=float64), 'ST': array([], dtype=float64)}}, 'Recall': {'m1': {'1dAV